In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
#export
from exp.imports import *

# Urls


In [3]:
#export
URL ='https://cloud.dfki.de/owncloud/index.php/s/FC2s2j23dHrPWxY'
LOCAL_PATH = Path.cwd()

In [4]:
#export

class URLs():
    "Global constants for dataset URLs."
    LOCAL_PATH = Path.cwd()
    BASE ='https://cloud.dfki.de/owncloud/index.php/s/FC2s2j23dHrPWxY/'
    LOG=f"{BASE}download?path=%2Fevent_logs&files="

    
    # BPI Challenges
    BPIC_2012  = f'{LOG}BPI_Challenge_2012.xes.gz'
    
    BPIC_2013_closed_problems  = f'{LOG}BPI_Challenge_2013_closed_problems.xes.gz'
    BPIC_2013_open_problems = f'{LOG}BPI_Challenge_2013_open_problems.xes.gz'
    BPIC_2013_incidents = f'{LOG}BPI_Challenge_2013_incidents.xes.gz'
   
    BPIC_2014_incidents_activity_log = f'{LOG}BPI_Challenge_2014_Detail_Incident_Activity.csv.gz'
    BPIC_2014_change_details = f'{LOG}BPI_Challenge_2014_Detail_Change.csv.gz'
    BPIC_2014_incident_details = f'{LOG}BPI_Challenge_2014_Detail_Incident.csv.gz'
    BPIC_2014_interaction_details = f'{LOG}BPI_Challenge_2014_Detail_Interaction.csv.gz'
   
    BPIC_2015_Municipality_1 = f'{LOG}BPIC15_1.xes.gz'
    BPIC_2015_Municipality_2 = f'{LOG}BPIC15_2.xes.gz'
    BPIC_2015_Municipality_3 = f'{LOG}BPIC15_3.xes.gz'
    BPIC_2015_Municipality_4 = f'{LOG}BPIC15_4.xes.gz'
    BPIC_2015_Municipality_5 = f'{LOG}BPIC15_5.xes.gz'
    
    BPIC_2016_Clicks_Logged_in = f'{LOG}BPI2016_Clicks_Logged_In.csv.gz'
    BPIC_2016_Clicks_Not_Logged_in = f'{LOG}BPI2016_Clicks_NOT_Logged_In.csv.gz'
    BPIC_2016_Complaints = f'{LOG}BPI2016_Complaints.csv.gz'
    BPIC_2016_Questions = f'{LOG}BPI2016_Questions.csv.gz'
    BPIC_2016_Werkmap_Messages = f'{LOG}BPI2016_Werkmap_Messages.csv.gz'
   
    BPIC_2017 = f'{LOG}BPI Challenge 2017.xes.gz'
    BPIC_2017_offer_log = f'{LOG}BPI Challenge 2017_offer_log.xes.gz'
    BPIC_2018 = f'{LOG}BPI Challenge 2018.xes.gz'
    BPIC_2019 = f'{LOG}BPI_Challenge_2019.xes.gz'
    
    
    # Process Discovery Contest
    PDC_2016_Training = 'https://www.win.tue.nl/ieeetfpm/lib/exe/fetch.php?media=shared:downloads:contest_dataset.zip'
    PDC_2016_Test = 'https://www.win.tue.nl/ieeetfpm/lib/exe/fetch.php?media=shared:contest_testeventlog.zip'
    PDC_2016_Models = 'https://www.win.tue.nl/ieeetfpm/lib/exe/fetch.php?media=shared:contest_models.zip'
    
    
    PDC_2017_Training = 'https://www.win.tue.nl/ieeetfpm/lib/exe/fetch.php?media=shared:dc_traning_logs_2017.zip'
    PDC_2017_Test =  'https://www.win.tue.nl/ieeetfpm/lib/exe/fetch.php?media=shared:test_log_contest_2017.zip'
    PDC_2017_Models = 'https://www.win.tue.nl/ieeetfpm/lib/exe/fetch.php?media=shared:models_contest_2017.zip'
    
    
    
    PDC_2019_Training = 'http://icpmconference.org/2019/wp-content/uploads/sites/6/2019/01/PDCtrainingLogs.zip'
    PDC_2019_Test1 = 'http://icpmconference.org/2019/wp-content/uploads/sites/6/2019/04/logFebruary.zip'
    PDC_2019_Test2 = 'http://icpmconference.org/2019/wp-content/uploads/sites/6/2019/04/log-15March.zip'
    

## Config Class

In [10]:
#export
def _expand_path(fpath): return Path(fpath).expanduser()

class Config():
    "Creates a default config file 'config.yml' in $FASTPM HOME (default `~/.fastpm/`)"
    DEFAULT_CONFIG_LOCATION = os.path.expanduser(os.getenv('FASTPM_HOME', '~/.fastpm'))
    DEFAULT_CONFIG_PATH = DEFAULT_CONFIG_LOCATION + '/config.yml'
    DEFAULT_CONFIG = {
        'data_path': DEFAULT_CONFIG_LOCATION + '/data'
    }
    
    @classmethod
    def get_key(cls, key):
        cf=cls.get()
        "Get the path to `key` in the config file."
        return cf.get(key, cls.DEFAULT_CONFIG.get(key,None))

    @classmethod
    def get_path(cls, path):
        "Get the `path` in the config file."
        return _expand_path(cls.get_key(path))

    @classmethod
    def data_path(cls):
        "Get the path to data in the config file."
        return cls.get_path('data_path')

    @classmethod
    def data_archive_path(cls):
        "Get the path to data archives in the config file."
        return cls.get_path('data_archive_path')


    @classmethod
    def get(cls, fpath=None, create_missing=True):
        "Retrieve the `Config` in `fpath`."
        fpath = _expand_path(fpath or cls.DEFAULT_CONFIG_PATH)
        if not fpath.exists() and create_missing: cls.create(fpath)
        assert fpath.exists(), f'Could not find config at: {fpath}. Please create'
        with open(fpath, 'r') as yaml_file:
            yfile=yaml.safe_load(yaml_file)
            return yfile

    @classmethod
    def create(cls, fpath):
        "Creates a `Config` from `fpath`."
        fpath = _expand_path(fpath)
        assert(fpath.suffix == '.yml')
        if fpath.exists(): return
        fpath.parent.mkdir(parents=True, exist_ok=True)
        with open(fpath, 'w') as yaml_file:
            yaml.dump(cls.DEFAULT_CONFIG, yaml_file, default_flow_style=False)


# Store Data sets on Disc

In [11]:
#export
def download_url(url:str, dest:str, fname:str=None, overwrite:bool=False, pbar:ProgressBar=None,
                 show_progress=True, chunk_size=1024*1024, timeout=4, retries=5)->None:
    "Download `url` to `dest` unless it exists and not `overwrite`."
    if os.path.exists(dest) and not overwrite: return

    s = requests.Session()
    s.mount('http://',requests.adapters.HTTPAdapter(max_retries=retries))
    u = s.get(url, stream=True, timeout=timeout)
    try: 
        file_size = int(u.headers["Content-Length"])
    except: show_progress = False
    
    with open(dest, 'wb') as f:
        nbytes = 0
        if show_progress: pbar = progress_bar(range(file_size), auto_update=False, leave=False, parent=pbar)
        try:
            for chunk in u.iter_content(chunk_size=chunk_size):
                nbytes += len(chunk)
                if show_progress: pbar.update(nbytes)
                f.write(chunk)
        except requests.exceptions.ConnectionError as e:
            fname = url.split('/')[-1]
            from fastai.datasets import Config
            data_dir = Config().data_path()
            timeout_txt =(f'\n Download of {url} has failed after {retries} retries\n'
                          f' Fix the download manually:\n'
                          f'$ mkdir -p {data_dir}\n'
                          f'$ cd {data_dir}\n'
                          f'$ wget -c {url}\n'
                          f'$ tar -zxvf {fname}\n\n'
                          f'And re-run your code once the download is successful\n')
            print(timeout_txt)
            import sys;sys.exit(1)


In [12]:
#export
def url2name(url):
    fname = url.split("/")[-1].split("=")[-1]
    return fname

def url2folder(url):
    fname = url.split("path=%2F")[-1].split("&")[0]
    return fname

def url2path(url):
    "Change `url` to a path."
    name = url2name(url)
    folder=url2folder(url)
    return datapath4file(name,folder) 


def datapath4file(filename,folder):
    "Return data path to `filename`, checking locally first then in the config file."
    local_path = URLs.LOCAL_PATH/folder
    if local_path.exists(): return local_path/filename
    else: return Config.data_path()/folder/filename

def download_data(url:str, fname:PathOrStr=None) -> Path:
    "Download `url` to destination `fname`."
    fname = Path(ifnone(fname, url2path(url)))
    os.makedirs(fname.parent, exist_ok=True)
    if not fname.exists():
        print(f'Downloading {url}')
        download_url(url, fname)
    return fname



In [13]:
url2path(URLs.BPIC_2012)

PosixPath('/home/lahann/.fastpm/data/event_logs/BPI_Challenge_2012.xes.gz')

In [14]:
download_data(URLs.BPIC_2012)

PosixPath('/home/lahann/.fastpm/data/event_logs/BPI_Challenge_2012.xes.gz')

In [15]:
download_data(URLs.BPIC_2013_closed_problems)

PosixPath('/home/lahann/.fastpm/data/event_logs/BPI_Challenge_2013_closed_problems.xes.gz')

In [16]:
download_data(URLs.BPIC_2013_incidents)

PosixPath('/home/lahann/.fastpm/data/event_logs/BPI_Challenge_2013_incidents.xes.gz')

In [17]:
download_data(URLs.BPIC_2013_open_problems)

PosixPath('/home/lahann/.fastpm/data/event_logs/BPI_Challenge_2013_open_problems.xes.gz')

In [18]:
download_data(URLs.BPIC_2014_change_details)

PosixPath('/home/lahann/.fastpm/data/event_logs/BPI_Challenge_2014_Detail_Change.csv.gz')

In [19]:
download_data(URLs.BPIC_2014_incident_details)

PosixPath('/home/lahann/.fastpm/data/event_logs/BPI_Challenge_2014_Detail_Incident.csv.gz')

In [20]:
download_data(URLs.BPIC_2014_incidents_activity_log)

PosixPath('/home/lahann/.fastpm/data/event_logs/BPI_Challenge_2014_Detail_Incident_Activity.csv.gz')

In [21]:
download_data(URLs.BPIC_2014_interaction_details)

PosixPath('/home/lahann/.fastpm/data/event_logs/BPI_Challenge_2014_Detail_Interaction.csv.gz')

In [22]:
download_data(URLs.BPIC_2015_Municipality_1)

PosixPath('/home/lahann/.fastpm/data/event_logs/BPIC15_1.xes.gz')

In [23]:
download_data(URLs.BPIC_2015_Municipality_2)

PosixPath('/home/lahann/.fastpm/data/event_logs/BPIC15_2.xes.gz')

In [24]:
download_data(URLs.BPIC_2015_Municipality_3)

PosixPath('/home/lahann/.fastpm/data/event_logs/BPIC15_3.xes.gz')

In [25]:
download_data(URLs.BPIC_2015_Municipality_4)

PosixPath('/home/lahann/.fastpm/data/event_logs/BPIC15_4.xes.gz')

In [26]:
download_data(URLs.BPIC_2015_Municipality_5)

PosixPath('/home/lahann/.fastpm/data/event_logs/BPIC15_5.xes.gz')

In [27]:
download_data(URLs.BPIC_2016_Clicks_Logged_in)

PosixPath('/home/lahann/.fastpm/data/event_logs/BPI2016_Clicks_Logged_In.csv.gz')

In [28]:
download_data(URLs.BPIC_2016_Clicks_Not_Logged_in)

PosixPath('/home/lahann/.fastpm/data/event_logs/BPI2016_Clicks_NOT_Logged_In.csv.gz')

In [29]:
download_data(URLs.BPIC_2016_Complaints)

PosixPath('/home/lahann/.fastpm/data/event_logs/BPI2016_Complaints.csv.gz')

In [30]:
download_data(URLs.BPIC_2016_Questions)

PosixPath('/home/lahann/.fastpm/data/event_logs/BPI2016_Questions.csv.gz')

In [31]:
download_data(URLs.BPIC_2016_Werkmap_Messages)

PosixPath('/home/lahann/.fastpm/data/event_logs/BPI2016_Werkmap_Messages.csv.gz')

In [32]:
download_data(URLs.BPIC_2017)

PosixPath('/home/lahann/.fastpm/data/event_logs/BPI Challenge 2017.xes.gz')

In [33]:
download_data(URLs.BPIC_2017_offer_log)

PosixPath('/home/lahann/.fastpm/data/event_logs/BPI Challenge 2017_offer_log.xes.gz')

In [34]:
download_data(URLs.BPIC_2018)

PosixPath('/home/lahann/.fastpm/data/event_logs/BPI Challenge 2018.xes.gz')

In [35]:
download_data(URLs.BPIC_2019)

PosixPath('/home/lahann/.fastpm/data/event_logs/BPI_Challenge_2019.xes.gz')

# Untar multiple formats

In [36]:
#export

def unzip(path):
    newpath=path.parent
    if not newpath.exists():
        print('unzip ...')
        with zipfile.ZipFile(path, 'r') as zip_ref:
            zip_ref.extractall(path.parent)
    return newpath

def gunzip(path):
    newpath=path.parent/path.stem
    if not newpath.exists():
        print('gunzip ...')
        with gzip.open(path, 'rb') as f_in:
            with open(newpath, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
    return newpath

_sup_compressions = {
    '.zip': unzip,
    '.gz': gunzip,
}
def untar_data(url:str, fname:PathOrStr=None) -> Path:
    path=download_data(url,fname)
    extract = _sup_compressions.get(path.suffix, False)
    if not extract: 
        print('not supported compression format')
        return path
    return extract(path)


In [37]:
untar_data(URLs.BPIC_2012)

PosixPath('/home/lahann/.fastpm/data/event_logs/BPI_Challenge_2012.xes')

In [38]:
untar_data(URLs.BPIC_2013_closed_problems)

PosixPath('/home/lahann/.fastpm/data/event_logs/BPI_Challenge_2013_closed_problems.xes')

In [39]:
untar_data(URLs.BPIC_2013_incidents)

PosixPath('/home/lahann/.fastpm/data/event_logs/BPI_Challenge_2013_incidents.xes')

In [40]:
untar_data(URLs.BPIC_2013_open_problems)

PosixPath('/home/lahann/.fastpm/data/event_logs/BPI_Challenge_2013_open_problems.xes')

In [41]:
untar_data(URLs.BPIC_2014_change_details)

PosixPath('/home/lahann/.fastpm/data/event_logs/BPI_Challenge_2014_Detail_Change.csv')

In [42]:
untar_data(URLs.BPIC_2014_incident_details)

PosixPath('/home/lahann/.fastpm/data/event_logs/BPI_Challenge_2014_Detail_Incident.csv')

In [43]:
untar_data(URLs.BPIC_2014_incidents_activity_log)

PosixPath('/home/lahann/.fastpm/data/event_logs/BPI_Challenge_2014_Detail_Incident_Activity.csv')

In [44]:
untar_data(URLs.BPIC_2014_interaction_details)

PosixPath('/home/lahann/.fastpm/data/event_logs/BPI_Challenge_2014_Detail_Interaction.csv')

In [45]:
untar_data(URLs.BPIC_2015_Municipality_1)

PosixPath('/home/lahann/.fastpm/data/event_logs/BPIC15_1.xes')

In [46]:
untar_data(URLs.BPIC_2015_Municipality_2)

PosixPath('/home/lahann/.fastpm/data/event_logs/BPIC15_2.xes')

In [47]:
untar_data(URLs.BPIC_2015_Municipality_3)

PosixPath('/home/lahann/.fastpm/data/event_logs/BPIC15_3.xes')

In [48]:
untar_data(URLs.BPIC_2015_Municipality_4)

PosixPath('/home/lahann/.fastpm/data/event_logs/BPIC15_4.xes')

In [49]:
untar_data(URLs.BPIC_2015_Municipality_5)

PosixPath('/home/lahann/.fastpm/data/event_logs/BPIC15_5.xes')

In [50]:
untar_data(URLs.BPIC_2016_Clicks_Logged_in)

PosixPath('/home/lahann/.fastpm/data/event_logs/BPI2016_Clicks_Logged_In.csv')

In [51]:
untar_data(URLs.BPIC_2016_Clicks_Not_Logged_in)

PosixPath('/home/lahann/.fastpm/data/event_logs/BPI2016_Clicks_NOT_Logged_In.csv')

In [52]:
untar_data(URLs.BPIC_2016_Complaints)

PosixPath('/home/lahann/.fastpm/data/event_logs/BPI2016_Complaints.csv')

In [53]:
untar_data(URLs.BPIC_2016_Questions)

PosixPath('/home/lahann/.fastpm/data/event_logs/BPI2016_Questions.csv')

In [54]:
untar_data(URLs.BPIC_2016_Werkmap_Messages)

PosixPath('/home/lahann/.fastpm/data/event_logs/BPI2016_Werkmap_Messages.csv')

In [55]:
untar_data(URLs.BPIC_2017)

PosixPath('/home/lahann/.fastpm/data/event_logs/BPI Challenge 2017.xes')

In [56]:
untar_data(URLs.BPIC_2017_offer_log)

PosixPath('/home/lahann/.fastpm/data/event_logs/BPI Challenge 2017_offer_log.xes')

In [57]:
untar_data(URLs.BPIC_2018)

gunzip ...


PosixPath('/home/lahann/.fastpm/data/event_logs/BPI Challenge 2018.xes')

In [58]:
untar_data(URLs.BPIC_2019)

PosixPath('/home/lahann/.fastpm/data/event_logs/BPI_Challenge_2019.xes')

# Export

In [59]:
! /home/lahann/anaconda3/envs/fastpm/bin/python notebook2script.py 01_fastpm_datasets.ipynb

Converted 01_fastpm_datasets.ipynb to exp/datasets.py


# Timeit Experiments with pandas

In [ ]:
import pandas as pd

In [ ]:
%%timeit -n 5
df = pd.DataFrame(columns=["A", "B", "C"]); new_row = pd.Series({"A": 4, "B": 4, "C": 4})
for i in range(160):
    df.loc[i] = new_row

In [ ]:
%%timeit -n 5

l=[]
for i in range(160):
    l.append([4,4,4])
pd.DataFrame(l,columns=["A", "B", "C"]);

In [ ]:
%%timeit -n 5

df = pd.DataFrame(columns=["A", "B", "C"]); new_row = pd.Series({"A": 4, "B": 4, "C": 4})
for i in range(160):
    df = df.append(new_row, ignore_index=True)

# 6.59 s ± 53.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
